In [7]:
import json
import numpy as np

In [8]:
def write_dict(path_json):
    """Write the keywords to a json file
    
    Using this approach I don't have to upload data base schema information to github.
    
    """
    
    data_schema = {'histogram_column' : 'source column name',
                   'lable_suffix' : 'suffix',
                   'lable' : 'target column name',
                   'table' : 'field_data',
                   'factor' : 1,
                   'bin_start' : 0,
                   'bin_end' : 10,
                   'bin_count' : 10,
                   'where': 'lala > 0',
                   'save_as': 'my_hist.sql'}
    
    with open(path_json, 'w') as f:
        f.write(json.dumps(data_schema))
    
    return None
                

def create_sql_histogram(data_schema):
    """Read the data base schema information from json and create sql_query"""
    
    bins = np.linspace(data_schema['bin_start'], 
                       data_schema['bin_end'],
                       num=data_schema['bin_count'],
                       endpoint=True)
    
    sql_query = ''
    for idx, _ in enumerate(bins[0:-1]):
        if idx == (len(bins) - 2):
            union = 'ORDER BY idx;'
        else:
            union = 'UNION'
            
        sql_query = """%sSELECT
    '%.2f to %.2f %s' AS Bucket,
    %d AS idx,
    COUNT(%s) * %.15f AS "%s"
FROM %s
WHERE %s BETWEEN %f AND %f
%s
%s
""" % (sql_query,
       bins[idx],
       bins[idx+1],
       data_schema['lable_suffix'],
       idx,
       data_schema['histogram_column'],
       data_schema['factor'],
       data_schema['lable'],
       data_schema['table'],
       data_schema['histogram_column'],
       bins[idx],
       bins[idx+1],
       data_schema['where'],
       union) 
        
        
    return sql_query

In [9]:
with open('vehicle_speed.json', 'r') as f:
    data_schema = json.load(f)

my_query = create_sql_histogram(data_schema)

print(my_query)

with open(data_schema['save_as'], 'w') as f:
    f.write(my_query)

SELECT
    '0.00 to 5.00 kph' AS Bucket,
    0 AS idx,
    COUNT(epsabsmaxvehspd) * 0.000002777777777 AS "Duration [h]"
FROM field_data
WHERE epsabsmaxvehspd BETWEEN 0.000000 AND 5.000000

UNION
SELECT
    '5.00 to 10.00 kph' AS Bucket,
    1 AS idx,
    COUNT(epsabsmaxvehspd) * 0.000002777777777 AS "Duration [h]"
FROM field_data
WHERE epsabsmaxvehspd BETWEEN 5.000000 AND 10.000000

UNION
SELECT
    '10.00 to 15.00 kph' AS Bucket,
    2 AS idx,
    COUNT(epsabsmaxvehspd) * 0.000002777777777 AS "Duration [h]"
FROM field_data
WHERE epsabsmaxvehspd BETWEEN 10.000000 AND 15.000000

UNION
SELECT
    '15.00 to 20.00 kph' AS Bucket,
    3 AS idx,
    COUNT(epsabsmaxvehspd) * 0.000002777777777 AS "Duration [h]"
FROM field_data
WHERE epsabsmaxvehspd BETWEEN 15.000000 AND 20.000000

UNION
SELECT
    '20.00 to 25.00 kph' AS Bucket,
    4 AS idx,
    COUNT(epsabsmaxvehspd) * 0.000002777777777 AS "Duration [h]"
FROM field_data
WHERE epsabsmaxvehspd BETWEEN 20.000000 AND 25.000000

UNION
SELECT
    